In [7]:
import psycopg2
import pandas as pd

from google.cloud import bigquery
from google.oauth2 import service_account

Conexion a posgrest y la base de datos

In [8]:
conn = psycopg2.connect( host = 'localhost', port = 5432, database = 'Adventureworks', user = 'postgres', password = '1234')

In [9]:
cur = conn.cursor()
table1 = pd.read_sql_query( 'SELECT * FROM production.product', conn )
table2 = pd.read_sql_query( 'SELECT * FROM production.productsubcategory', conn )
table3 = pd.read_sql_query( 'SELECT * FROM production.productcategory', conn )
table4 = pd.read_sql_query( 'SELECT * FROM production.productcosthistory', conn )
table5 = pd.read_sql_query( 'SELECT * FROM sales.salesorderheader', conn )
table6 = pd.read_sql_query( 'SELECT * FROM sales.customer', conn )
table7 = pd.read_sql_query( 'SELECT *  FROM sales.salesterritory', conn )
table8 = pd.read_sql_query( 'SELECT *  FROM sales.salesorderdetail', conn )
table9 = pd.read_sql_query( 'SELECT *  FROM sales.specialofferproduct', conn )
table10= pd.read_sql_query('SELECT * FROM sales.specialoffer', conn )
## Tablas de consultas SQL
table11= pd.read_sql_query('select purchasing.vendor.name AS nombreVendedor, humanresources.employee.jobtitle AS tipoEmpleado, purchasing.productvendor.productid AS idproducto from purchasing.vendor, humanresources.employee, purchasing.productvendor, purchasing.purchaseorderheader where   purchasing.vendor.businessentityid = purchasing.productvendor.businessentityid and  purchasing.purchaseorderheader.vendorid = purchasing.vendor.businessentityid and  purchasing.purchaseorderheader.employeeid = humanresources.employee.businessentityid Group by (nombreVendedor,tipoEmpleado,idproducto)', conn )
table12= pd.read_sql_query( 'select sales.salesterritory.name AS territorio,sales.store.name AS tienda,sales.salespersonquotahistory.salesquota AS cuotaVentas from  sales.salesterritory,sales.store,sales.salesperson, sales.salespersonquotahistory where sales.store.salespersonid = sales.salesperson.businessentityid and sales.salesperson.territoryid = sales.salesterritory.territoryid and sales.salesperson.businessentityid = sales.salespersonquotahistory.businessentityid GROUP BY(territorio,tienda,cuotaVentas)', conn )
cur.close()

In [10]:
conn.close()

<span style="color:purple">UNION DE LAS TABLAS PARA LA BODEGA DE DATOS</span>

In [11]:
result = pd.merge(table1[['productid','name','productsubcategoryid']], table4[['productid','standardcost']], on = 'productid')
result.head(5)

,productid,name,productsubcategoryid,standardcost
0,707,"Sport-100 Helmet, Red",31.0,12.0278
1,707,"Sport-100 Helmet, Red",31.0,13.8782
2,707,"Sport-100 Helmet, Red",31.0,13.0863
3,708,"Sport-100 Helmet, Black",31.0,12.0278
4,708,"Sport-100 Helmet, Black",31.0,13.8782


In [73]:
result1= pd.merge(result, table2[['productsubcategoryid','productcategoryid','name']],  on = 'productsubcategoryid')
result1

,productid,name_x,productsubcategoryid,standardcost,productcategoryid,name_y
0,707,"Sport-100 Helmet, Red",31.0,12.0278,4,Helmets
1,707,"Sport-100 Helmet, Red",31.0,13.8782,4,Helmets
2,707,"Sport-100 Helmet, Red",31.0,13.0863,4,Helmets
3,708,"Sport-100 Helmet, Black",31.0,12.0278,4,Helmets
4,708,"Sport-100 Helmet, Black",31.0,13.8782,4,Helmets
...,...,...,...,...,...,...
390,978,"Touring-3000 Blue, 44",3.0,461.4448,1,Touring Bikes
391,979,"Touring-3000 Blue, 50",3.0,461.4448,1,Touring Bikes
392,994,LL Bottom Bracket,5.0,23.9716,2,Bottom Brackets
393,995,ML Bottom Bracket,5.0,44.9506,2,Bottom Brackets


In [74]:
result2 = pd.merge(result1,table3[['productcategoryid','name']],on = 'productcategoryid')
result2

,productid,name_x,productsubcategoryid,standardcost,productcategoryid,name_y,name
0,707,"Sport-100 Helmet, Red",31.0,12.0278,4,Helmets,Accessories
1,707,"Sport-100 Helmet, Red",31.0,13.8782,4,Helmets,Accessories
2,707,"Sport-100 Helmet, Red",31.0,13.0863,4,Helmets,Accessories
3,708,"Sport-100 Helmet, Black",31.0,12.0278,4,Helmets,Accessories
4,708,"Sport-100 Helmet, Black",31.0,13.8782,4,Helmets,Accessories
...,...,...,...,...,...,...,...
390,970,"Touring-2000 Blue, 46",3.0,755.1508,1,Touring Bikes,Bikes
391,971,"Touring-2000 Blue, 50",3.0,755.1508,1,Touring Bikes,Bikes
392,972,"Touring-2000 Blue, 54",3.0,755.1508,1,Touring Bikes,Bikes
393,978,"Touring-3000 Blue, 44",3.0,461.4448,1,Touring Bikes,Bikes


In [75]:
grouping = result2.groupby( result2[ 'productid']).agg( { 'standardcost': sum } ).reset_index().rename( columns = { 'standardcost': 'sum' } ).sort_values( by = 'productid' )

In [76]:
grouping

,productid,sum
0,707,38.9923
1,708,38.9923
2,709,3.3963
3,710,3.3963
4,711,38.9923
...,...,...
288,995,44.9506
289,996,53.9416
290,997,343.6496
291,998,343.6496


<span style="color:purple">PRIMERA TABLA DE LA BODEGA DE DATOS</span>

In [77]:
result3=pd.merge(result2,grouping, on='productid')
Tabla1=result3[['name_x','name_y','name','sum']].rename( columns = { 'name_x': 'nombre_producto','name_y':'subcategoria','name':'categoria','sum':'suma_totalXproducto' } ).drop_duplicates()

In [78]:
Tabla1

,nombre_producto,subcategoria,categoria,suma_totalXproducto
0,"Sport-100 Helmet, Red",Helmets,Accessories,38.9923
3,"Sport-100 Helmet, Black",Helmets,Accessories,38.9923
6,"Sport-100 Helmet, Blue",Helmets,Accessories,38.9923
9,"Touring-Panniers, Large",Panniers,Accessories,51.5625
10,Cable Lock,Locks,Accessories,10.3125
...,...,...,...,...
390,"Touring-2000 Blue, 46",Touring Bikes,Bikes,755.1508
391,"Touring-2000 Blue, 50",Touring Bikes,Bikes,755.1508
392,"Touring-2000 Blue, 54",Touring Bikes,Bikes,755.1508
393,"Touring-3000 Blue, 44",Touring Bikes,Bikes,461.4448


<span style="color:purple">UNION DE LAS TABLAS PARA LA BODEGA DE DATOS</span>

In [79]:
resul=pd.merge(table1[['productid','name']],table9[['productid','specialofferid']], on = 'productid')
resul

,productid,name,specialofferid
0,680,"HL Road Frame - Black, 58",1
1,706,"HL Road Frame - Red, 58",1
2,707,"Sport-100 Helmet, Red",1
3,707,"Sport-100 Helmet, Red",2
4,707,"Sport-100 Helmet, Red",3
...,...,...,...
533,998,"Road-750 Black, 48",1
534,998,"Road-750 Black, 48",2
535,998,"Road-750 Black, 48",3
536,999,"Road-750 Black, 52",1


In [80]:
resul1=pd.merge(resul[['productid','specialofferid','name']],table10[['specialofferid','type']], on = 'specialofferid')
resul1

,productid,specialofferid,name,type
0,680,1,"HL Road Frame - Black, 58",No Discount
1,706,1,"HL Road Frame - Red, 58",No Discount
2,707,1,"Sport-100 Helmet, Red",No Discount
3,708,1,"Sport-100 Helmet, Black",No Discount
4,709,1,"Mountain Bike Socks, M",No Discount
...,...,...,...,...
533,963,13,"Touring-3000 Yellow, 54",New Product
534,964,13,"Touring-3000 Yellow, 58",New Product
535,965,13,"Touring-3000 Yellow, 62",New Product
536,978,13,"Touring-3000 Blue, 44",New Product


In [81]:
resul2=pd.merge(resul1,table8[['productid','salesorderid']], on = 'productid')
resul2

,productid,specialofferid,name,type,salesorderid
0,707,1,"Sport-100 Helmet, Red",No Discount,43665
1,707,1,"Sport-100 Helmet, Red",No Discount,43668
2,707,1,"Sport-100 Helmet, Red",No Discount,43673
3,707,1,"Sport-100 Helmet, Red",No Discount,43677
4,707,1,"Sport-100 Helmet, Red",No Discount,43678
...,...,...,...,...,...
296978,999,2,"Road-750 Black, 52",Volume Discount,74008
296979,999,2,"Road-750 Black, 52",Volume Discount,74010
296980,999,2,"Road-750 Black, 52",Volume Discount,74028
296981,999,2,"Road-750 Black, 52",Volume Discount,74094


In [82]:
resul3=pd.merge(resul2,table5[['salesorderid','orderdate','customerid']], on = 'salesorderid')
resul3

,productid,specialofferid,name,type,salesorderid,orderdate,customerid
0,707,1,"Sport-100 Helmet, Red",No Discount,43665,2011-05-31,29580
1,707,2,"Sport-100 Helmet, Red",Volume Discount,43665,2011-05-31,29580
2,707,3,"Sport-100 Helmet, Red",Volume Discount,43665,2011-05-31,29580
3,707,8,"Sport-100 Helmet, Red",Seasonal Discount,43665,2011-05-31,29580
4,707,11,"Sport-100 Helmet, Red",Seasonal Discount,43665,2011-05-31,29580
...,...,...,...,...,...,...,...
296978,999,2,"Road-750 Black, 52",Volume Discount,73704,2014-05-24,23885
296979,999,1,"Road-750 Black, 52",No Discount,73926,2014-05-27,24387
296980,999,2,"Road-750 Black, 52",Volume Discount,73926,2014-05-27,24387
296981,999,1,"Road-750 Black, 52",No Discount,74008,2014-05-28,21386


In [83]:
resul4=pd.merge(resul3,table6[['customerid','territoryid']], on = 'customerid')
resul4

,productid,specialofferid,name,type,salesorderid,orderdate,customerid,territoryid
0,707,1,"Sport-100 Helmet, Red",No Discount,43665,2011-05-31,29580,1
1,707,2,"Sport-100 Helmet, Red",Volume Discount,43665,2011-05-31,29580,1
2,707,3,"Sport-100 Helmet, Red",Volume Discount,43665,2011-05-31,29580,1
3,707,8,"Sport-100 Helmet, Red",Seasonal Discount,43665,2011-05-31,29580,1
4,707,11,"Sport-100 Helmet, Red",Seasonal Discount,43665,2011-05-31,29580,1
...,...,...,...,...,...,...,...,...
296978,999,2,"Road-750 Black, 52",Volume Discount,73704,2014-05-24,23885,4
296979,999,1,"Road-750 Black, 52",No Discount,73926,2014-05-27,24387,4
296980,999,2,"Road-750 Black, 52",Volume Discount,73926,2014-05-27,24387,4
296981,999,1,"Road-750 Black, 52",No Discount,74008,2014-05-28,21386,4


<span style="color:purple">SEGUNDA TABLA DE LA BODEGA DE DATOS</span>

In [84]:
resul5=pd.merge(resul4,table7[['territoryid','name']], on = 'territoryid')
Tabla2=resul5[['name_x','type','orderdate','name_y']].rename( columns = { 'name_x': 'nombre_producto','type':'tipo_oferta','orderdate':'fecha_orden','name_y':'territorio' } ).drop_duplicates()
Tabla2

,nombre_producto,tipo_oferta,fecha_orden,territorio
0,"Sport-100 Helmet, Red",No Discount,2011-05-31,Northwest
1,"Sport-100 Helmet, Red",Volume Discount,2011-05-31,Northwest
3,"Sport-100 Helmet, Red",Seasonal Discount,2011-05-31,Northwest
5,"Mountain Bike Socks, M",No Discount,2011-05-31,Northwest
6,"Mountain Bike Socks, M",Volume Discount,2011-05-31,Northwest
...,...,...,...,...
296972,"Road-750 Black, 52",Volume Discount,2013-11-13,Australia
296973,"Road-750 Black, 52",No Discount,2013-12-01,Australia
296974,"Road-750 Black, 52",Volume Discount,2013-12-01,Australia
296981,"Road-750 Black, 52",No Discount,2014-05-08,Australia


<span style="color:purple">TERCERA TABLA DE LA BODEGA DE DATOS</span>

In [126]:
table11.columns=['tipoEmpleado','Vendedor','idproducto']
Tabla3 = table11
Tabla3

,tipoEmpleado,Vendedor,idproducto
0,Allenson Cycles,Purchasing Assistant,530
1,Hill Bicycle Center,Purchasing Assistant,524
2,Aurora Bike Center,Purchasing Assistant,405
3,Cruger Bike Company,Purchasing Assistant,375
4,Leaf River Terrain,Purchasing Manager,436
...,...,...,...
1280,Australia Bike Retailer,Purchasing Manager,431
1281,Advanced Bicycles,Purchasing Assistant,359
1282,Norstan Bike Hut,Purchasing Assistant,381
1283,Cruger Bike Company,Buyer,395


<span style="color:purple">CUARTA TABLA DE LA BODEGA DE DATOS</span>

In [133]:
table12.columns=['territorio','tienda','cuotaVentas']
Tabla4=table12
Tabla4

,territorio,tienda,cuotaVentas
0,United Kingdom,Mail Market,1279000.0
1,Southwest,Economy Bikes Company,1276000.0
2,Central,Responsible Bike Dealers,971000.0
3,Canada,First-Rate Outlet,356000.0
4,Northwest,Year-Round Sports,280000.0
...,...,...,...
7373,Australia,Nationwide Supply,389000.0
7374,Southeast,Parts Shop,917000.0
7375,Southwest,Bicycle Merchandise Warehouse,555000.0
7376,Australia,Eastside Cycle Shop,389000.0


<span style="color:green ">CARGA DE LAS TABLAS A BIG QUERY</span>

In [44]:
credentials = service_account.Credentials.from_service_account_file( 
    'C:/Users/USUARIO/Documents/Trabajos Yerson/Semestre 6/Base de Datos 2/TareaAdventureworks/adventureworks-315120-80fd5417fb17.json', 
    scopes = [ 'https://www.googleapis.com/auth/cloud-platform' ]
)

In [45]:
client = bigquery.Client( credentials = credentials, project = credentials.project_id )

In [46]:
job_config = bigquery.LoadJobConfig(
    schema = [
        # Supported datatypes: https://cloud.google.com/bigquery/docs/reference/standard-sql/data-types
        bigquery.SchemaField( 'nombre_producto', bigquery.enums.SqlTypeNames.STRING ),
        bigquery.SchemaField( 'subcategoria', bigquery.enums.SqlTypeNames.STRING ),
        bigquery.SchemaField( 'categoria', bigquery.enums.SqlTypeNames.STRING ),
        bigquery.SchemaField( 'suma_totalXproducto', bigquery.enums.SqlTypeNames.FLOAT64),
    ],
    # Drod and re-create table, if exist
    write_disposition = 'WRITE_TRUNCATE',
)

In [47]:
job_config = bigquery.LoadJobConfig(
    schema = [
        # Supported datatypes: https://cloud.google.com/bigquery/docs/reference/standard-sql/data-types
        bigquery.SchemaField( 'nombre_producto', bigquery.enums.SqlTypeNames.STRING ),
        bigquery.SchemaField( 'tipo_oferta', bigquery.enums.SqlTypeNames.STRING ),
        bigquery.SchemaField( 'fecha_orden', bigquery.enums.SqlTypeNames.DATE ),
        bigquery.SchemaField( 'territorio', bigquery.enums.SqlTypeNames.STRING),
    ],
    # Drod and re-create table, if exist
    write_disposition = 'WRITE_TRUNCATE',
)

In [48]:
job_config = bigquery.LoadJobConfig(
    schema = [
        # Supported datatypes: https://cloud.google.com/bigquery/docs/reference/standard-sql/data-types
        bigquery.SchemaField( 'tipoEmpleado', bigquery.enums.SqlTypeNames.STRING),
        bigquery.SchemaField( 'Vendedor', bigquery.enums.SqlTypeNames.STRING),
        bigquery.SchemaField( 'idproducto', bigquery.enums.SqlTypeNames.INT64 ),
    ],
    # Drod and re-create table, if exist
    write_disposition = 'WRITE_TRUNCATE',
)

In [136]:
job_config = bigquery.LoadJobConfig(
    schema = [
        # Supported datatypes: https://cloud.google.com/bigquery/docs/reference/standard-sql/data-types
        bigquery.SchemaField( 'territorio', bigquery.enums.SqlTypeNames.STRING),
        bigquery.SchemaField( 'tienda', bigquery.enums.SqlTypeNames.STRING),
        bigquery.SchemaField( 'cuotaVentas', bigquery.enums.SqlTypeNames.FLOAT64 ),
    ],
    # Drod and re-create table, if exist
    write_disposition = 'WRITE_TRUNCATE',
)

In [58]:
job = client.load_table_from_dataframe(
    Tabla1, 'DataAdventureworkd.Tabla1', job_config = job_config
)
job.result()

In [98]:
job = client.load_table_from_dataframe(
    Tabla2, 'DataAdventureworkd.Tabla2', job_config = job_config
)
job.result()

In [49]:
job = client.load_table_from_dataframe(
    Tabla3, 'DataAdventureworkd.Tabla3', job_config = job_config
)
job.result()

In [137]:
job = client.load_table_from_dataframe(
    Tabla4, 'DataAdventureworkd.Tabla4', job_config = job_config
)
job.result()

In [59]:
table = client.get_table( 'DataAdventureworkd.Tabla1' )
print(
    'Loaded {} rows and {} columns to {}'.format(
        table.num_rows, len( table.schema ), 'DataAdventureworkd.Tabla1'
    )
)

Loaded 293 rows and 4 columns to DataAdventureworkd.Tabla1


In [99]:
table = client.get_table( 'DataAdventureworkd.Tabla2' )
print(
    'Loaded {} rows and {} columns to {}'.format(
        table.num_rows, len( table.schema ), 'DataAdventureworkd.Tabla2'
    )
)

Loaded 120806 rows and 4 columns to DataAdventureworkd.Tabla2


In [50]:
table = client.get_table( 'DataAdventureworkd.Tabla3' )
print(
    'Loaded {} rows and {} columns to {}'.format(
        table.num_rows, len( table.schema ), 'DataAdventureworkd.Tabla3'
    )
)

Loaded 1285 rows and 3 columns to DataAdventureworkd.Tabla3


In [138]:
table = client.get_table( 'DataAdventureworkd.Tabla4' )
print(
    'Loaded {} rows and {} columns to {}'.format(
        table.num_rows, len( table.schema ), 'DataAdventureworkd.Tabla4'
    )
)

Loaded 7378 rows and 3 columns to DataAdventureworkd.Tabla4
